In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/Thesis/reimp_lore")

import os
os.chdir('/content/drive/MyDrive/Thesis/reimp_lore')

# Now the directory '?' is the current working directory.

In [3]:
!pip install bitarray

In [4]:
!pip install scikit-multilearn

In [5]:
!pip install arff

In [6]:
import lore
from datamanager import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, accuracy_score

from util import record2str

ipynb to py in pycharm:
- jupyter nbconvert --to script new_run.ipynb

## Dataset

In [7]:
## Iris Dataset
# dataset_name = 'dataset/iris.csv'
# dataset = prepare_iris_dataset(dataset_name)

## wine
# dataset_name = 'dataset/wine.csv'
# dataset = prepare_wine_dataset(dataset_name)

##############################################
#           Categorical dataset              #
##############################################
## german: (0 = Good, 1 = Bad)
# dataset_name = 'dataset/german_credit.csv'
# dataset = prepare_german_dataset(dataset_name)

## adult: ['<=50K', '>50K']
# dataset_name = 'dataset/adult.csv'
# dataset = prepare_adult_dataset(dataset_name)

## compas-scores-two-years: ['High', 'Low', 'Medium']
dataset_name = 'dataset/compas-scores-two-years.csv'
dataset = prepare_compass_dataset(dataset_name)

dataframe = dataset[0]
class_name = dataset[1]
dataset_fin = prepare_dataset(dataframe, class_name)

In [8]:
df = dataset_fin[0] #dataframe with unique numeric class values(0, 1, ...)
feature_names = dataset_fin[1]
class_values = dataset_fin[2]
numeric_columns = dataset_fin[3]
rdf = dataset_fin[4] #real dataframe
real_feature_names = dataset_fin[5]
features_map = dataset_fin[6] #map each class name to its unique numeric value
df_categorical_idx = dataset_fin[7]

In [9]:
rdf.head()

,age,priors_count,days_b_screening_arrest,is_recid,is_violent_recid,two_year_recid,length_of_stay,age_cat,sex,race,c_charge_degree,class
0,69,0,1,0,0,0,0,Greater than 45,Male,Other,F,Low
1,34,0,1,1,1,1,10,25 - 45,Male,African-American,F,Low
2,24,4,1,1,0,1,1,Less than 25,Male,African-American,F,Low
3,23,1,1,0,0,0,0,Less than 25,Male,African-American,F,High
4,43,2,1,0,0,0,0,25 - 45,Male,Other,F,Low


In [10]:
df.head()

,age,priors_count,days_b_screening_arrest,is_recid,is_violent_recid,two_year_recid,length_of_stay,age_cat=25 - 45,age_cat=Greater than 45,age_cat=Less than 25,sex=Female,sex=Male,race=African-American,race=Asian,race=Caucasian,race=Hispanic,race=Native American,race=Other,c_charge_degree=F,c_charge_degree=M,class
0,69,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1
1,34,0,1,1,1,1,10,1,0,0,0,1,1,0,0,0,0,0,1,0,1
2,24,4,1,1,0,1,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
3,23,1,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0
4,43,2,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,1


## Black box classifier

In [11]:
X = df.loc[:, df.columns != class_name].values
y = df[class_name].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
blackbox = RandomForestClassifier()
blackbox.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [12]:
y_pred = blackbox.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.598


## select an instance _x_

In [13]:
i = 10
x = X_test[i]
y_val = blackbox.predict(x.reshape(1,-1))[0]

print(class_values)
class_prob = blackbox.predict_proba(x.reshape(1,-1))[0]
print(class_prob)

y_val_name = class_values[y_val]
print('blackbox(x) = { %s }' % y_val_name)


['High', 'Low', 'Medium']
[0.   0.94 0.06]
blackbox(x) = { Low }


In [14]:
print('x = %s' % record2str(x, feature_names, numeric_columns))

x = { age = 33, priors_count = 0, days_b_screening_arrest = 1, is_recid = 0, is_violent_recid = 0, two_year_recid = 0, length_of_stay = 0, age_cat = 25 - 45, sex = Female, race = Caucasian, c_charge_degree = M }


# LORE explainer (explaining an instance x)

In [15]:
lore_obj = lore.LORE(X_test, blackbox, feature_names, class_name, class_values,
                 numeric_columns, features_map, df_categorical_idx, neigh_type='ngmusx', verbose=False)

In [16]:
# just to check
# Z = lore_obj.neighgen_fn(x, categorical_columns=df_categorical_idx)
# print('Z is:',Z)
# Z.shape

In [17]:
explanation = lore_obj.explain_instance(x, samples=1000, nbr_runs=10, exemplar_num=3)
print(explanation)

r = { age_cat != Less than 25 } --> { class: Low }

feature importance = {'age_cat=Less than 25': 0.890237819899506}

c = { { age_cat = Less than 25 } --> { class: High } }

exemplars = { age = 33, priors_count = 0, days_b_screening_arrest = 1, is_recid = 0, is_violent_recid = 0, two_year_recid = 0, length_of_stay = 0, age_cat = 25 - 45, sex = Male, race = Caucasian, c_charge_degree = M }
{ age = 32, priors_count = 0, days_b_screening_arrest = 1, is_recid = 0, is_violent_recid = 0, two_year_recid = 0, length_of_stay = 1, age_cat = 25 - 45, sex = Female, race = Caucasian, c_charge_degree = M }
{ age = 33, priors_count = 0, days_b_screening_arrest = 1, is_recid = 1, is_violent_recid = 0, two_year_recid = 1, length_of_stay = 0, age_cat = 25 - 45, sex = Female, race = Caucasian, c_charge_degree = M }

counter-exemplars = { age = 24, priors_count = 0, days_b_screening_arrest = 0, is_recid = 0, is_violent_recid = 0, two_year_recid = 0, length_of_stay = 0, age_cat = Less than 25, sex = Female

# X-Bot

In [18]:
!ls

chatdata		    lore.py			 rule.py
classes.pkl		    neighborgen_lore.py		 text_preprocessing.py
datamanager.py		    new_run.ipynb		 util.py
dataset			    new_run.py			 words.pkl
decision_tree.py	    __pycache__			 xbot_model.h5
DeepLearning-Chatbot.ipynb  README.md			 xbot_model.py
explanation.py		    response_generator_keras.py
__init__.py		    response_generator.py


### Model

In [19]:
from response_generator_keras import ResponseGeneratorKeras
from xbot_model import XBotModel
from keras.models import load_model

import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
###################### Chat Data ######################
data_file = open('chatdata/intents.json').read()
data = json.loads(data_file)

In [21]:
XBot_obj = XBotModel(verbose=True)

In [22]:
# train dataset
train_dataset = XBot_obj.prepare_train_dataset(data)
train_x = list(train_dataset[:, 0])
train_y = list(train_dataset[:, 1])

Training data created!


In [23]:
# create model
model = XBot_obj.create_model(train_x, train_y, dropout=0.5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               12800     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 21,706
Trainable params: 21,706
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# compile and fit the model
mymodel = XBot_obj.compile_fit_model(model, train_x, train_y, epochs=100, 
                                     batch_size=5,
                                     lr=5e-3, 
                                     loss='categorical_crossentropy')

Epoch 1/100
10/10 [==============================] - 1s 2ms/step - loss: 2.2814 - accuracy: 0.1897
Epoch 2/100
10/10 [==============================] - 0s 2ms/step - loss: 2.1963 - accuracy: 0.1532
Epoch 3/100
10/10 [==============================] - 0s 2ms/step - loss: 1.9334 - accuracy: 0.3736
Epoch 4/100
10/10 [==============================] - 0s 2ms/step - loss: 1.8800 - accuracy: 0.3650
Epoch 5/100
10/10 [==============================] - 0s 2ms/step - loss: 1.5451 - accuracy: 0.5700
Epoch 6/100
10/10 [==============================] - 0s 2ms/step - loss: 1.3805 - accuracy: 0.6883
Epoch 7/100
10/10 [==============================] - 0s 2ms/step - loss: 1.0835 - accuracy: 0.7559
Epoch 8/100
10/10 [==============================] - 0s 2ms/step - loss: 0.8606 - accuracy: 0.7329
Epoch 9/100
10/10 [==============================] - 0s 3ms/step - loss: 0.8273 - accuracy: 0.8202
Epoch 10/100
10/10 [==============================] - 0s 2ms/step - loss: 0.6898 - accuracy: 0.8951
Epoch 11/

In [25]:
# save the model
model.save('xbot_model.h5', mymodel)

### Chatbot

In [26]:
# load model
model = load_model('xbot_model.h5')

In [27]:
xbot_response = ResponseGeneratorKeras(data, explanation, model)

xbot_response.start()

Hello, I'm X-Bot!
If you want to exit, type Bye!
hi
X-Bot: Good to see you again


how are you
X-Bot: Good to see you again


What help you provide?
X-Bot: I can give you an explanation consisting of a decision rule, explaining the factual reasons of the decision, and a set of counterfactuals, suggesting the changes in the instance features that would lead to a different outcome


How often does the system make mistakes?
X-Bot: The fidelity is: 
1.0


how accurate you are
X-Bot: Hello, thanks for asking


How accurate are the predictions?
X-Bot: The fidelity is: 
1.0


explain this instance for me
X-Bot: The factual rule are: 
{ age_cat != Less than 25 } --> { class: Low }


explain it to me
X-Bot: The factual rule are: 
{ age_cat != Less than 25 } --> { class: Low }


How should this feature change for this instance to get a different prediction?
X-Bot: The counterfactual rules are: 
{ { age_cat = Less than 25 } --> { class: High } }


top important variables
X-Bot: The most important